# torch.cuda

In [1]:
import torch
torch.cuda.memory_allocated()

0

In [2]:
torch.cuda.device_count()

1

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name()

'GeForce GTX 1080'

In [5]:
a = torch.rand(10, 10).cuda()
torch.cuda.memory_allocated() # Returns the current GPU memory occupied by tensors in bytes for a given device.

512

In [6]:
b = torch.rand(10, 1000).cuda()
torch.cuda.memory_allocated()

40960

In [7]:
c = torch.rand(10, 100000).cuda()
torch.cuda.memory_allocated()

4041216

In [8]:
del c
torch.cuda.ipc_collect()
torch.cuda.memory_allocated() 


40960

In [9]:
del b
torch.cuda.memory_allocated()

512

In [10]:
del a
torch.cuda.memory_allocated()

0

In [11]:
torch.cuda.max_memory_allocated(device=None) # Returns the maximum GPU memory occupied by tensors in bytes for a given device.

4041216

In [12]:
test2 = torch.Tensor(9000,100000).cuda()
torch.cuda.memory_allocated()

3600809984

In [13]:
test3 = torch.Tensor(2000,100000).cuda()
torch.cuda.memory_allocated()

4400809984

In [14]:
torch.cuda.max_memory_allocated(device=None)

4400809984

In [16]:
torch.cuda.max_memory_cached(device=None) 

4424990720

In [17]:
torch.cuda.memory_allocated()

4400809984